# Workflow
we will start with Data Collection first from Kaggle and as we have imported the required Data we shall then move on to EDA

During EDA(Exploratory Data Analytics) and this is a neccessary step towards a Machine Learning project since it helps with understanding the data and analyzing what models can be used to work on the data.

Then we move on to Data PreProcessing where we perform under sampling and over sampling.

Train Test Split - here we train and test the data
Then work with tree Models like random forest, xtree boost classifier, etc. as well as use cross validation.

Find the best trained model
Then send unknown data and predict of our required goal i.e to find out if a customer will be leaving or not.

Then we study the dataset and analyze what are the columns shown in it

now we download the required pytohn libraries for the prediction project
 do this by running these commands individually
NOTE: run each command only after the previous has stopped running.
Commands:

pip install pandas \n

pip install numpy 

pip install scikit-learn

pip install matplotlib 

pip install xgboost
after this restart the python  kernel being used in the notebook.>

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
# from imblearn import SMOTE #for oversampling technique to build a uniformly distributed target class
import seaborn as sns
from sklearn.model_selection import train_test_split #from splitting data into train test and split
from imblearn.over_sampling import SMOTE #to solve class imbalance issues

# lets import few decision tree based models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

In [ ]:
# now we load the csv data to a pandas DataFrame
df= pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.shape #this shows the number of rows and columns we have in our DataSet and this is a decent number of of data we have for training a model

(7043, 21)

In [ ]:
pd.set_option("display.max_columns", None) #this command ensures all columns are visible in the DataSet
df.head #this prints the first 5 columns of the dataset